In [ ]:
from larcv import larcv
import ROOT

import matplotlib
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
from matplotlib import pyplot as plt
import numpy as np

from ROOT import geo2d
pygeo = geo2d.PyDraw()

%matplotlib inline

In [ ]:
INFILE="/Users/vgenty/Desktop/intrinsic_nue/out_pid_merge_mask3/larcv_pid_0000_0099.root"

In [ ]:
cfg="""
ProcessDriver:{
  Verbosity: 0
  EnableFilter: false
  RandomAccess: false
  ProcessType: ["ChannelMax","ChannelMax","ChannelMax","CombineImages","SegmentMask","SegmentMask","MaskImage","MaskImage","LArbysImageExtract"]
  ProcessName: ["ChannelMaxPlane0","ChannelMaxPlane1","ChannelMaxPlane2","CombineImages","ShowerSegment","TrackSegment","ShowerImage","TrackImage","LArbysImageExtract"]
  AnaFile: "/tmp/_.root"
  IOManager: {
    Verbosity: 0
    Name:         "IOManager"
    IOMode:        2
    OutFileName:   "/tmp/__.root"
    StoreOnlyType: [0]
    StoreOnlyName: ["_"]
  }
  ProcessList: {
  ChannelMaxPlane0: {
      Verbosity: 2
      InProducer: "uburn_plane0"
      NPlanes: 2
      OutputProducer: "uburn_max_plane0"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }
   ChannelMaxPlane1: {
      Verbosity: 2
      InProducer: "uburn_plane1"
      NPlanes: 2
      OutputProducer: "uburn_max_plane1"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }
  ChannelMaxPlane2: {
      Verbosity: 2
      InProducer: "uburn_plane2"
      NPlanes: 2
      OutputProducer: "uburn_max_plane2"
      ChannelWeights: [1,1]
      RelevantChannels: [0,1]
      ChannelMask: [1,2]
   }                                                                                                                     
   CombineImages: {
      Verbosity: 2
      ImageProducers: ["uburn_max_plane0","uburn_max_plane1","uburn_max_plane2"]
      NPlanes: 1
      OutputProducer: "uburn"
    }
    ShowerSegment: {
      Verbosity: 2
      ROIProducer: ""
      ImageProducer:  "uburn"
      OutputProducer: "shower_segment"
      ValidROI: ["Cosmic"]  
      MaskValue: 0
      PlaneID: [0,1,2]
    }
  TrackSegment: {
      Verbosity: 2
      ROIProducer: ""
      ImageProducer:  "uburn"
      OutputProducer: "track_segment"
      ValidROI: ["BNB"]
      MaskValue: 0
      PlaneID: [0,1,2]
    }
    ShowerImage: {
      Verbosity: 2
      MinPIThreshold: 1
      MaskValue: 0
      OutputProducer: "shower_tpc"
      ReferenceProducer: "shower_segment"
      TargetProducer: "tpc"
    }
    TrackImage: {
      Verbosity: 2
      MinPIThreshold: 1
      MaskValue: 0
      OutputProducer: "track_tpc"
      ReferenceProducer: "track_segment"
      TargetProducer: "tpc"
    }
    LArbysImageExtract: {
      ADCImageProducer: "tpc"
      TrackImageProducer: "track_tpc"
      ShowerImageProducer: "shower_tpc"
      LArbysImageMaker: {
        Q2Gray: 2
        QMax: 500
        QMin: 0
      }
    }
  }
}
"""
with open("/tmp/cfg.txt",'w+') as f: f.write(cfg)

In [ ]:
proc=larcv.ProcessDriver('ProcessDriver')
proc.configure('/tmp/cfg.txt')
proc.override_input_file(ROOT.std.vector("string")(1,ROOT.std.string(INFILE)))
proc.initialize()  
iom=proc.io()
ext_id  = proc.process_id("LArbysImageExtract")                                                                                                                          
ext_proc = proc.process_ptr(ext_id) 

In [ ]:
event=0
if iom.current_entry()==event:
    proc.process_entry(event+1)
proc.process_entry(event)

In [ ]:
def segment_image(meta_v):
    img_v=[None,None,None]
    
    for plane in xrange(3):
        trackimg2d  = ext_proc._ev_trk.Image2DArray()[plane]                                                                                                           
        showerimg2d = ext_proc._ev_shr.Image2DArray()[plane]  

        trackimg2d=trackimg2d.crop(meta_v[plane])
        showerimg2d=showerimg2d.crop(meta_v[plane])

        trackimg = ext_proc.maker().ExtractMat(trackimg2d)
        showerimg = ext_proc.maker().ExtractMat(showerimg2d)

        timg=pygeo.image(trackimg)                                                                                                                                   
        simg=pygeo.image(showerimg)                                                                                                                                  

        timg=np.where(timg>10,200,0.0)                                                                                                                               
        simg=np.where(simg>10,100,0.0)                                                                                                                               
        img_v[plane]=simg.copy()+timg.copy()
        img_v[plane]=img_v[plane].T[::-1,:]
    return img_v

In [ ]:

evroi=iom.get_data(larcv.kProductROI,"croimerge").ROIArray()
evpix=iom.get_data(larcv.kProductPixel2D,"test_ctor")

for roi_id,roi in enumerate(evroi):
    print roi_id,")",roi
    
    img_v=segment_image(roi.BB())       
    
    for plane in xrange(3):
        fig,ax=plt.subplots(figsize=(10,12))
        ax.imshow(img_v[plane],cmap='jet',vmin=0,vmax=255,interpolation='none')
        ax.set_title("Plane {}".format(plane),fontsize=30)
        plt.show()
    print "<===================================================>"

In [ ]:
colors=['magenta','white','yellow']


evpgraph = iom.get_data(larcv.kProductPGraph,"test")
evctor   = iom.get_data(larcv.kProductPixel2D,"test_ctor")

for vertex_id, pgraph in enumerate(evpgraph.PGraphArray()):
    print vertex_id,pgraph
    pararray=pgraph.ParticleArray()
    
    img_v = segment_image(pararray.back().BB())
    ctor_v=[]
    for parid,id_ in enumerate(np.array(pgraph.ClusterIndexArray())):
        pix2d_vv=[]
        for plane in xrange(3):
            pix2d_v=evctor.Pixel2DClusterArray()[plane][id_]
            pix2d_v=[[pix2d_v[i_].X(),pix2d_v[i_].Y()] for i_ in xrange(pix2d_v.size())]
            pix2d_vv.append(np.array(pix2d_v))
        ctor_v.append(pix2d_vv)
        
    for plane in xrange(3):
        fig,ax=plt.subplots(figsize=(20,22))
        ax.imshow(img_v[plane],cmap='jet',vmin=0,vmax=255,interpolation='none')
        for ix,ctor in enumerate(ctor_v):
            ctor=ctor[plane]
            ax.plot(ctor[:,0],ctor[:,1],'-o',lw=3,color=colors[ix],alpha=1.0)



        ax.set_title("Plane {}".format(plane),fontsize=30)
        plt.show()